# Preprocessing

Preprocessing notebook

In [1]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = "plotly"

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_ipcw
from sksurv.util import Surv

df = pd.read_csv("../../data/clinical_train.csv")
df_eval = pd.read_csv("../../data/clinical_val.csv")
maf_df = pd.read_csv("../../data/molecular_train.csv")
maf_eval = pd.read_csv("../../data/molecular_val.csv")
target_df = pd.read_csv("../../data/target_train.csv")

In [12]:
maf_df2 = pd.read_csv("../../data/molecular_train.csv")
maf_eval2 = pd.read_csv("../../data/molecular_val.csv")

In [2]:
target = ['OS_YEARS', 'OS_STATUS']

# Drop rows where 'OS_YEARS' is NaN if conversion caused any issues
target_df.dropna(subset=target, inplace=True)
target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')
target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

# Select features
features = ['ID', 'BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT', 'CYTOGENETICS']

# Create the survival data format
X = df.loc[df['ID'].isin(target_df['ID']), features]
X_eval = df_eval.loc[:, features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)


In [3]:
df.columns

Index(['ID', 'CENTER', 'BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT',
       'CYTOGENETICS'],
      dtype='object')

## XGBoost-based Imputation

Create an XGBoost imputer to fill missing values using OS_YEARS as the prediction target. The imputer will be trained on complete cases and used to fill missing values in both training and evaluation datasets.

In [4]:
num_cols = X.select_dtypes("number").columns
print(f"Numerical columns are: {num_cols.values}")

o = len(X)
X = X[X[num_cols].isna().sum(axis=1) <= 4]
print(f"Dropping every row with more than 4 missing numerical values. {o - len(X)} rows dropped, {len(X)} remaining.")


Numerical columns are: ['BM_BLAST' 'WBC' 'ANC' 'MONOCYTES' 'HB' 'PLT']
Dropping every row with more than 4 missing numerical values. 70 rows dropped, 3103 remaining.


In [5]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
print(f"Fitting RobustScaler for numerical columns ({num_cols.values}) on training data, transforming training and evaluation data.")
X[num_cols] = scaler.fit_transform(X[num_cols])
X_eval[num_cols] = scaler.transform(X_eval[num_cols])


Fitting RobustScaler for numerical columns (['BM_BLAST' 'WBC' 'ANC' 'MONOCYTES' 'HB' 'PLT']) on training data, transforming training and evaluation data.


In [6]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

# Identify columns with missing values (only numerical columns)
missing_cols = [col for col in num_cols if X[col].isnull().any()]
print(f"Columns with missing values: {missing_cols}")

# For each column with missing values, train separate XGBoost model
xgb_imputers = {}

for col in missing_cols:
    print(f"\nTraining XGBoost imputer for {col}...")

    # Get rows where current column is not null
    col_complete_mask = X[col].notnull()
    X_train_col = X.loc[col_complete_mask, num_cols].drop(columns=[col])
    y_train_col = X.loc[col_complete_mask, col]

    # Handle any remaining missing values in other columns for training
    X_train_col = X_train_col.fillna(X_train_col.median())


    # Define objective function for Optuna
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'subsample': trial.suggest_float('subsample', 0.2, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
            'random_state': 42,
            'objective': 'reg:squarederror'
        }

        model = XGBRegressor(**params)
        scores = cross_val_score(model, X_train_col, y_train_col,
                                 cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        return -scores.mean()


    # Run Optuna optimization
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=250, show_progress_bar=True)

    print(f"Best parameters for {col}: {study.best_params}")
    print(f"Best CV score for {col}: {study.best_value:.4f}")

    # Train final model with best parameters
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['objective'] = 'reg:squarederror'

    best_model = XGBRegressor(**best_params)
    best_model.fit(X_train_col, y_train_col)

    # Store best estimator
    xgb_imputers[col] = best_model

[I 2025-12-05 22:52:31,233] A new study created in memory with name: no-name-7b925358-a701-4b44-acfd-7acaef31e19d


Columns with missing values: ['BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT']

Training XGBoost imputer for BM_BLAST...


  0%|          | 0/250 [00:00<?, ?it/s]

[I 2025-12-05 22:52:35,223] Trial 0 finished with value: 1.310970700728492 and parameters: {'max_depth': 5, 'learning_rate': 0.22648248189516848, 'n_estimators': 380, 'subsample': 0.6789267873576292, 'colsample_bytree': 0.3248149123539492, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 1.310970700728492.
[I 2025-12-05 22:52:37,896] Trial 1 finished with value: 1.1428864264017942 and parameters: {'max_depth': 8, 'learning_rate': 0.001124579825911934, 'n_estimators': 487, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 1 with value: 1.1428864264017942.
[I 2025-12-05 22:52:40,382] Trial 2 finished with value: 1.0439805689017168 and parameters: {'max_depth': 6, 'learning_rate': 0.005265139631677754, 'n_estimators': 325, 's

[I 2025-12-05 22:53:40,123] A new study created in memory with name: no-name-35651d38-dd0d-4da7-a35c-7ab6019f112f


...

[I 2025-12-05 22:56:21,330] Trial 0 finished with value: 0.6957071435978067 and parameters: {'max_depth': 5, 'learning_rate': 0.22648248189516848, 'n_estimators': 380, 'subsample': 0.6789267873576292, 'colsample_bytree': 0.3248149123539492, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 0.6957071435978067.
[I 2025-12-05 22:56:21,652] Trial 1 finished with value: 0.6027720205117575 and parameters: {'max_depth': 8, 'learning_rate': 0.001124579825911934, 'n_estimators': 487, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 1 with value: 0.6027720205117575.
[I 2025-12-05 22:56:21,838] Trial 2 finished with value: 0.5741731141410493 and parameters: {'max_depth': 6, 'learning_rate': 0.005265139631677754, 'n_estimators': 325, 

[I 2025-12-05 22:57:22,331] A new study created in memory with name: no-name-e01ab1b0-4323-426f-8cdf-f174bf58ce9a



Training XGBoost imputer for PLT...


  0%|          | 0/250 [00:00<?, ?it/s]

[I 2025-12-05 22:57:22,580] Trial 0 finished with value: 0.9221293931327503 and parameters: {'max_depth': 5, 'learning_rate': 0.22648248189516848, 'n_estimators': 380, 'subsample': 0.6789267873576292, 'colsample_bytree': 0.3248149123539492, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 0.9221293931327503.
[I 2025-12-05 22:57:22,940] Trial 1 finished with value: 0.7949200946036987 and parameters: {'max_depth': 8, 'learning_rate': 0.001124579825911934, 'n_estimators': 487, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 1 with value: 0.7949200946036987.
[I 2025-12-05 22:57:23,128] Trial 2 finished with value: 0.7269480200713755 and parameters: {'max_depth': 6, 'learning_rate': 0.005265139631677754, 'n_estimators': 325, 

In [7]:
def impute_missing_values(df, imputers, numerical_cols, dataset_name="dataset"):
    """
    Apply XGBoost imputation to missing values in a dataframe.
    
    Parameters:
    - df: DataFrame to impute
    - imputers: Dictionary of trained XGBoost imputers per column
    - numerical_cols: List of numerical column names to impute
    - dataset_name: Name for logging purposes
    
    Returns:
    - DataFrame with imputed values
    """
    missing_cols = [col for col in imputers.keys() if col in df.columns]

    for col in missing_cols:
        if df[col].isnull().sum() == 0:
            continue

        print(f"\nImputing {col} in {dataset_name}...")

        missing_mask = df[col].isnull()
        X_pred = df.loc[missing_mask, numerical_cols].drop(columns=[col])
        X_pred = X_pred.fillna(df[numerical_cols].drop(columns=[col]).median())

        predictions = imputers[col].predict(X_pred)
        df.loc[missing_mask, col] = predictions

        print(f"  Imputed {missing_mask.sum()} missing values")

    print(f"\nMissing values in {dataset_name} after imputation:")
    print(df[numerical_cols].isnull().sum())

    return df

In [8]:
X = impute_missing_values(X, xgb_imputers, num_cols, "X")
X_eval = impute_missing_values(X_eval, xgb_imputers, num_cols, "X_eval")



Imputing BM_BLAST in X...
  Imputed 27 missing values

Imputing WBC in X...
  Imputed 159 missing values

Imputing ANC in X...
  Imputed 71 missing values

Imputing MONOCYTES in X...
  Imputed 474 missing values

Imputing HB in X...
  Imputed 5 missing values

Imputing PLT in X...
  Imputed 16 missing values

Missing values in X after imputation:
BM_BLAST     0
WBC          0
ANC          0
MONOCYTES    0
HB           0
PLT          0
dtype: int64

Imputing BM_BLAST in X_eval...
  Imputed 115 missing values

Imputing WBC in X_eval...
  Imputed 112 missing values

Imputing ANC in X_eval...
  Imputed 141 missing values

Imputing MONOCYTES in X_eval...
  Imputed 883 missing values

Imputing HB in X_eval...
  Imputed 111 missing values

Imputing PLT in X_eval...
  Imputed 115 missing values

Missing values in X_eval after imputation:
BM_BLAST     0
WBC          0
ANC          0
MONOCYTES    0
HB           0
PLT          0
dtype: int64


## Enhanced Mutation Features

Computing mutation count (Nmut) along with VAF statistics (average, std, max) and length statistics (average, std, max) for both training and evaluation datasets.

Looking at the user request and the selected cell, I need to apply RobustScaler to the new mutation features (Nmut, VAF_avg, VAF_std, VAF_max, LEN_avg, LEN_std, LEN_max) that are computed in the `compute_mutation_features` function.

The task is to:
1. Apply RobustScaler to the newly computed mutation features after merging them with the dataframe
2. Fit the scaler on training data and transform both training and evaluation data



In [13]:
maf_df["CHR"] = maf_df2["CHR"]
maf_eval["CHR"] = maf_eval2["CHR"]

In [9]:
maf_df["CHR"].value_counts()[maf_df["CHR"].value_counts() > 90].index

Index(['4', '2', '17', 'X', '20', '21', '12', '7', '5', '1', '11', '15', '19',
       '18', '9', '3', '16'],
      dtype='object', name='CHR')

In [14]:
maf_df["EFFECT"].unique()

array(['non_synonymous_codon', 'stop_gained', 'splice_site_variant',
       'frameshift_variant', 'inframe_codon_loss', 'inframe_codon_gain',
       'initiator_codon_change', '2KB_upstream_variant',
       'complex_change_in_transcript', '3_prime_UTR_variant', 'stop_lost',
       'inframe_variant', 'synonymous_codon', 'stop_retained_variant',
       'ITD', 'PTD'], dtype=object)

In [17]:
pd.get_dummies(maf_df, columns=["EFFECT"])
pd.get_dummies(maf_eval, columns=["EFFECT"])

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,VAF,DEPTH,EFFECT_ITD,EFFECT_PTD,EFFECT_frameshift_variant,EFFECT_inframe_codon_gain,EFFECT_inframe_codon_loss,EFFECT_non_synonymous_codon,EFFECT_stop_gained,EFFECT_stop_lost
0,KYW961,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.2620,485.0,False,False,False,False,False,True,False,False
1,KYW142,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.0280,527.0,False,False,False,False,False,True,False,False
2,KYW453,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.2920,277.0,False,False,False,False,False,True,False,False
3,KYW982,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.0970,821.0,False,False,False,False,False,True,False,False
4,KYW845,1,36932209.0,36932209.0,G,A,CSF3R,p.Q754X,0.4300,358.0,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,KYW1077,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.4231,NaN,False,True,False,False,False,False,False,False
3085,KYW1084,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.0176,NaN,False,True,False,False,False,False,False,False
3086,KYW1082,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.2273,NaN,False,True,False,False,False,False,False,False
3087,KYW1085,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.2941,NaN,False,True,False,False,False,False,False,False


In [18]:
pd.get_dummies(maf_df, columns=["CHR"])
pd.get_dummies(maf_eval, columns=["CHR"])

,ID,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH,...,CHR_21,CHR_22,CHR_3,CHR_4,CHR_5,CHR_6,CHR_7,CHR_8,CHR_9,CHR_X
0,KYW961,1747229.0,1747229.0,T,C,GNB1,p.K57E,non_synonymous_codon,0.2620,485.0,...,False,False,False,False,False,False,False,False,False,False
1,KYW142,1747229.0,1747229.0,T,C,GNB1,p.K57E,non_synonymous_codon,0.0280,527.0,...,False,False,False,False,False,False,False,False,False,False
2,KYW453,1747229.0,1747229.0,T,C,GNB1,p.K57E,non_synonymous_codon,0.2920,277.0,...,False,False,False,False,False,False,False,False,False,False
3,KYW982,1747229.0,1747229.0,T,C,GNB1,p.K57E,non_synonymous_codon,0.0970,821.0,...,False,False,False,False,False,False,False,False,False,False
4,KYW845,36932209.0,36932209.0,G,A,CSF3R,p.Q754X,stop_gained,0.4300,358.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,KYW1077,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,0.4231,NaN,...,False,False,False,False,False,False,False,False,False,False
3085,KYW1084,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,0.0176,NaN,...,False,False,False,False,False,False,False,False,False,False
3086,KYW1082,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,0.2273,NaN,...,False,False,False,False,False,False,False,False,False,False
3087,KYW1085,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,0.2941,NaN,...,False,False,False,False,False,False,False,False,False,False


In [21]:
maf_df["CHR_14"]

KeyError: 'CHR_14'

Looking at the user request, I need to add CHR count columns for the chromosomes specified in the list: ['4', '2', '17', 'X', '20', '21', '12', '7', '5', '1', '11', '15', '19', '18', '9', '3', '16'].

Currently, the code only has `CHR_X_count`. I need to add similar count columns for all the other chromosomes in the list.

Here's the modified cell code:



In [19]:
maf_df["CHR"].unique()

array(['11', '5', '3', '4', '2', '22', '17', 'X', '12', '9', '7', '1',
       '8', '16', '20', '21', '19', '15', '13', '6', '18', '14', '10',
       nan], dtype=object)

In [22]:
from sklearn.preprocessing import RobustScaler

def compute_mutation_features(maf_df, X_df):
    """Compute mutation statistics and merge with features dataframe."""
    maf_df = maf_df.copy()

    # Compute length for each mutation
    maf_df['LEN'] = maf_df['END'] - maf_df['START'] + 1
    maf_df['DELLEN'] = maf_df['LEN'] - maf_df['REF'].apply(lambda x: len(str(x)))

    # Valeurs uniques de CHR (hors NaN)
    unique_chr = sorted(maf_df['CHR'].dropna().unique())

    # Dictionnaire pour l'aggregation nommée
    agg_dict = {
        'Nmut': ('ID', 'size'),
        'VAF_avg': ('VAF', 'mean'),
        'VAF_std': ('VAF', 'std'),
        'VAF_max': ('VAF', 'max'),
        'LEN_avg': ('LEN', 'mean'),
        'LEN_max': ('LEN', 'max'),
        'DELLEN_sum': ('DELLEN', 'sum'),
        'DEPTH_avg': ('DEPTH', 'mean'),
        'DEPTH_std': ('DEPTH', 'std'),
        'DEPTH_max': ('DEPTH', 'max'),
        'DEPTH_min': ('DEPTH', 'min'),
        'CHR_nunique': ('CHR', 'nunique'),
        'EFFECT_nunique': ('EFFECT', 'nunique'),
        'EFFECT_FV_count': ('EFFECT', lambda x: (x == 'frameshift_variant').sum()),
        'EFFECT_SG_count': ('EFFECT', lambda x: (x == 'stop_gained').sum()),
        'EFFECT_NS_count': ('EFFECT', lambda x: (x == 'non_synonymous_codon').sum()),
    }

    # Ajout dynamique des colonnes CHR_*_count
    for ch in unique_chr:
        col_name = f'CHR_{ch}_count'
        # val=ch pour "figer" la valeur dans la lambda
        agg_dict[col_name] = ('CHR', lambda x, val=ch: (x == val).sum())

    # Agrégation
    tmp = maf_df.groupby('ID').agg(**agg_dict).reset_index()

    # Fill NaN std values with 0 (occurs when patient has only 1 mutation)
    tmp[['VAF_std', 'DEPTH_std']] = tmp[['VAF_std', 'DEPTH_std']].fillna(0)

    # Merge with X
    X_w_mutation = X_df.merge(tmp, on='ID', how='left')

    # Colonnes ajoutées par l'agg (à remplir avec 0 pour les patients sans mutation)
    new_cols = [c for c in tmp.columns if c != 'ID']
    X_w_mutation[new_cols] = X_w_mutation[new_cols].fillna(0)

    return X_w_mutation


# On suppose que compute_mutation_features est déjà défini comme avant
X_w_mutation = compute_mutation_features(maf_df, X)
X_eval_w_mutation = compute_mutation_features(maf_eval, X_eval)

# Construction de la liste des features de mutation (à partir du train)
base_mutation_features = [
    'Nmut', 'VAF_avg', 'VAF_std', 'VAF_max',
    'LEN_avg', 'LEN_max', 'DELLEN_sum',
    'DEPTH_avg', 'DEPTH_std', 'DEPTH_max', 'DEPTH_min',
    'CHR_nunique',
    'EFFECT_nunique', 'EFFECT_FV_count', 'EFFECT_SG_count', 'EFFECT_NS_count'
]

# Toutes les colonnes de type CHR_*_count dans le TRAIN
chr_count_cols = [
    c for c in X_w_mutation.columns
    if c.startswith('CHR_') and c.endswith('_count')
]

mutation_features = base_mutation_features + chr_count_cols

# 🔧 Harmoniser les colonnes entre train et eval :
# si une colonne de mutation_features manque, on la crée avec des 0
for col in mutation_features:
    if col not in X_w_mutation.columns:
        X_w_mutation[col] = 0
    if col not in X_eval_w_mutation.columns:
        X_eval_w_mutation[col] = 0

# (Optionnel mais propre) réordonner les colonnes
X_w_mutation = X_w_mutation.copy()
X_eval_w_mutation = X_eval_w_mutation.copy()
X_w_mutation[mutation_features] = X_w_mutation[mutation_features]
X_eval_w_mutation[mutation_features] = X_eval_w_mutation[mutation_features]

# RobustScaler sur ces features
mutation_scaler = RobustScaler()
print(
    f"Fitting RobustScaler for mutation features ({mutation_features}) "
    "on training data, transforming training and evaluation data."
)
X_w_mutation[mutation_features] = mutation_scaler.fit_transform(X_w_mutation[mutation_features])
X_eval_w_mutation[mutation_features] = mutation_scaler.transform(X_eval_w_mutation[mutation_features])


Fitting RobustScaler for mutation features (['Nmut', 'VAF_avg', 'VAF_std', 'VAF_max', 'LEN_avg', 'LEN_max', 'DELLEN_sum', 'DEPTH_avg', 'DEPTH_std', 'DEPTH_max', 'DEPTH_min', 'CHR_nunique', 'EFFECT_nunique', 'EFFECT_FV_count', 'EFFECT_SG_count', 'EFFECT_NS_count', 'CHR_1_count', 'CHR_10_count', 'CHR_11_count', 'CHR_12_count', 'CHR_13_count', 'CHR_14_count', 'CHR_15_count', 'CHR_16_count', 'CHR_17_count', 'CHR_18_count', 'CHR_19_count', 'CHR_2_count', 'CHR_20_count', 'CHR_21_count', 'CHR_22_count', 'CHR_3_count', 'CHR_4_count', 'CHR_5_count', 'CHR_6_count', 'CHR_7_count', 'CHR_8_count', 'CHR_9_count', 'CHR_X_count']) on training data, transforming training and evaluation data.


In [23]:
X_w_mutation

,ID,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,Nmut,VAF_avg,...,CHR_21_count,CHR_22_count,CHR_3_count,CHR_4_count,CHR_5_count,CHR_6_count,CHR_7_count,CHR_8_count,CHR_9_count,CHR_X_count
0,P132697,1.571429,-0.3250,-0.666667,0.513219,-0.807692,-0.024316,"46,xy,del(20)(q12)[2]/46,xy[18]",2.000000,-0.131147,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P132698,-0.285714,0.8250,0.148148,-0.419907,0.730769,-0.492401,"46,xx",0.000000,-0.042407,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P116889,1.714286,-0.1000,0.037037,-0.419907,1.730769,-0.255319,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]",0.000000,-1.015868,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,P132699,-0.285714,-0.0500,-0.037037,-0.419907,-0.307692,-0.279635,"46,xy,del(3)(q26q27)[15]/46,xy[5]",2.666667,-0.307681,...,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0
4,P132700,0.428571,30.9750,2.851852,0.824261,0.538462,0.437690,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]",-0.666667,0.788078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,P121826,-0.285714,-0.4000,-0.362963,-0.264386,0.192308,-0.273556,"47~49,xy,+21,+21,+22[cp11]/46,xy[10]",0.333333,-0.475365,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3099,P121827,-0.214286,1.0000,0.244444,0.124417,0.615385,-0.504559,"44,xx,inv(2)(p25q31),-4,del(5)(q12q33),-18[20]",-0.333333,0.602168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3100,P121830,-0.428571,-0.5750,-0.537037,-0.124417,-0.115385,-0.224924,"46,xy,del(20)(q11.2q13.1)[4]/45,xy,idem,-7[16]",1.000000,-0.160025,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
3101,P121853,0.285714,-0.6825,-0.603704,-0.404355,0.653846,-0.127660,"46,xx,del(1)(p34)[5]/45,xx,sl,-18[12]/46,xx,sd...",0.333333,-0.411797,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


# Processing Cytogenetics

sorte de "rules engine" basé sur la littérature clinique AML pour extraire des features pronostiques clés depuis l’ISCN. On détecte

* **Monosomal karyotype (MK)**: au moins 2 monosomies autosomiques, ou 1 monosomie autosomique + au moins une anomalie structurale. C’est franchement mauvais pronostic. ([PubMed Central][1])
* **Complex karyotype**: typiquement ≥ 3 anomalies cytogénétiques indépendantes, pronostic défavorable. ([Cancer Network][2])
* **Chromosomes 5 et 7**: perte de 5 ou 7, ou del(5q)/del(7q), classés défavorables dans ELN 2022. ([ASH Publications][3])
* **+8 (trisomie 8)**: très fréquente, plutôt risque intermédiaire en AML. ([MDPI][4])
* **Translocations favorables classiques**: t(8;21), inv(16)/t(16;16); APL t(15;17) est à part. On les isole comme features car elles portent un signal fort. ([ASH Publications][3])
* **Proportions clonales** via les crochets `[n]` pour calculer la part de métaphases portant une anomalie donnée. La sémantique des notations vient de l’ISCN. ([PubMed][5])

Pour chaque karyotype, on construit:

* `has_any_abnormality`, `n_events`, `n_chromosomes_altered`
* `has_minus5_or_del5q`, `has_minus7_or_del7q`, `has_plus8`
* `has_t_8_21`, `has_inv16_or_t_16_16`, `has_t_15_17`
* `is_complex_karyotype`, `is_monosomal_karyotype`
* `total_metaphases` et des **proportions clonales**: `prop_any_abnormal`, `prop_adverse_5_7`, `prop_plus8`, etc.
* Option: `eln_like_flag_adverse_cyto` basé ici sur 5/7 ou MK ou complex.

[1]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3069222/?utm_source=chatgpt.com "Acute myeloid leukemia with monosomal karyotype at the ..."
[2]: https://www.cancernetwork.com/view/unfavorable-complex-and-monosomal-karyotypes-most-challenging-forms-acute-myeloid-leukemia?utm_source=chatgpt.com "Unfavorable, Complex, and Monosomal Karyotypes"
[3]: https://ashpublications.org/blood/article/140/12/1345/485817/Diagnosis-and-management-of-AML-in-adults-2022?utm_source=chatgpt.com "Diagnosis and management of AML in adults - ASH Publications"
[4]: https://www.mdpi.com/2072-6694/13/22/5679?utm_source=chatgpt.com "Risk Stratification, Measurable Residual Disease, and ..."
[5]: https://pubmed.ncbi.nlm.nih.gov/34839499/?utm_source=chatgpt.com "[Introduction and interpretation of the updated contents of ..."


In [24]:
import re
from typing import List, Dict

# --- REGEX patterns ---
_ISCN_EVENT_RE = re.compile(r'(del|dup|inv|ins|i|t|add|der)\s*\(', re.IGNORECASE)
_MONOSOMY_RE   = re.compile(r'(?<![pq])-(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)
_TRISOMY_RE    = re.compile(r'(?<![pq])\+(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)
_CHR_NUM_RE    = re.compile(r'(?<![pq])(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)

# specific adverse/favorable
_MINUS5_OR_DEL5Q_RE = re.compile(r'-(?:5)(?![pq])|del\s*\(\s*5\s*\)\s*\(\s*q', re.IGNORECASE)
_MINUS7_OR_DEL7Q_RE = re.compile(r'-(?:7)(?![pq])|del\s*\(\s*7\s*\)\s*\(\s*q', re.IGNORECASE)
_PLUS8_RE           = re.compile(r'\+8(?![pq])', re.IGNORECASE)
_T_8_21_RE          = re.compile(r't\s*\(\s*8\s*;\s*21\s*\)', re.IGNORECASE)
_INV16_OR_T_16_16_RE= re.compile(r'(inv\s*\(\s*16\s*\)|t\s*\(\s*16\s*;\s*16\s*\))', re.IGNORECASE)
_T_15_17_RE         = re.compile(r't\s*\(\s*15\s*;\s*17\s*\)', re.IGNORECASE)
_STRUCTURAL_RE      = re.compile(r'(del|dup|inv|ins|i|t|add|der)\s*\(', re.IGNORECASE)

# --- helpers ---
def _split_clones(karyo: str) -> List[str]:
    return [c.strip() for c in str(karyo).split('/') if c.strip()]

def _extract_metaphases(clone: str) -> int:
    m = re.search(r'\[(\d+)\]', clone)
    return int(m.group(1)) if m else 0

def _count_events(clone: str) -> int:
    n_struct = len(_ISCN_EVENT_RE.findall(clone))
    n_mono   = len(_MONOSOMY_RE.findall(clone))
    n_tri    = len(_TRISOMY_RE.findall(clone))
    n_mono_minusY = len(re.findall(r'(?<![pq])-(?:Y)(?![pq])', clone, flags=re.IGNORECASE))
    return n_struct + n_tri + max(n_mono - n_mono_minusY, 0)

def _chromosomes_altered(clone: str) -> int:
    nums = set()
    for m in _MONOSOMY_RE.finditer(clone):
        nums.add(m.group(1).upper())
    for m in _TRISOMY_RE.finditer(clone):
        nums.add(m.group(1).upper())
    for ev in re.finditer(r'(del|dup|inv|ins|i|t|add|der)\s*\(([^)]+)\)', clone, flags=re.IGNORECASE):
        for x in re.split(r'[;,\s]+', ev.group(2)):
            if _CHR_NUM_RE.fullmatch(x.strip()):
                nums.add(x.strip().upper())
    nums.discard('Y')
    return len(nums)

def _has_structural(clone: str) -> bool:
    return bool(_STRUCTURAL_RE.search(clone))

def _autosomic_monosomies(clone: str) -> List[int]:
    return [int(m.group(1)) for m in _MONOSOMY_RE.finditer(clone) if m.group(1).upper() not in ('X','Y')]

def _is_monosomal_karyotype(karyo: str) -> bool:
    clones = _split_clones(karyo)
    autosomal_monosomies = set()
    any_struct = False
    for c in clones:
        autosomal_monosomies.update(_autosomic_monosomies(c))
        any_struct = any_struct or _has_structural(c)
    return (len(autosomal_monosomies) >= 2) or (len(autosomal_monosomies) >= 1 and any_struct)

def _is_complex_karyotype(karyo: str) -> bool:
    clones = _split_clones(karyo)
    total_events = 0
    for c in clones:
        c_wo_minusY = re.sub(r'(?<![pq])-(?:Y)(?![pq])', '', c, flags=re.IGNORECASE)
        total_events += _count_events(c_wo_minusY)
    return total_events >= 3

def _clone_flags(clone: str) -> Dict[str, bool]:
    return {
        'minus5_or_del5q': bool(_MINUS5_OR_DEL5Q_RE.search(clone)),
        'minus7_or_del7q': bool(_MINUS7_OR_DEL7Q_RE.search(clone)),
        'plus8':           bool(_PLUS8_RE.search(clone)),
        't_8_21':          bool(_T_8_21_RE.search(clone)),
        'inv16_or_t_16_16':bool(_INV16_OR_T_16_16_RE.search(clone)),
        't_15_17':         bool(_T_15_17_RE.search(clone)),
        'has_structural':  _has_structural(clone),
        'events_count':    _count_events(clone),
        'chrs_altered':    _chromosomes_altered(clone),
        'has_any_abn':     bool(_ISCN_EVENT_RE.search(clone) or _MONOSOMY_RE.search(clone) or _TRISOMY_RE.search(clone)),
    }

# --- main featurizer ---
def add_cytogenetics_features(df: pd.DataFrame, col: str = "CYTOGENETICS") -> pd.DataFrame:
    rows = []
    for k in df[col]:
        clones = _split_clones(k)
        clone_info = []
        total_meta_known = 0
        for c in clones:
            n_meta = _extract_metaphases(c)
            flags = _clone_flags(c)
            clone_info.append((c, n_meta, flags))
            total_meta_known += n_meta

        any_abn = any(f['has_any_abn'] for _,_,f in clone_info)
        n_events = sum(f['events_count'] for _,_,f in clone_info)
        n_chrs   = max([f['chrs_altered'] for _,_,f in clone_info] + [0])

        has_minus5_or_del5q = any(f['minus5_or_del5q'] for _,_,f in clone_info)
        has_minus7_or_del7q = any(f['minus7_or_del7q'] for _,_,f in clone_info)
        has_plus8           = any(f['plus8'] for _,_,f in clone_info)
        has_t_8_21          = any(f['t_8_21'] for _,_,f in clone_info)
        has_inv16_or_t_16_16= any(f['inv16_or_t_16_16'] for _,_,f in clone_info)
        has_t_15_17         = any(f['t_15_17'] for _,_,f in clone_info)

        is_mk  = _is_monosomal_karyotype(k)
        is_ck  = _is_complex_karyotype(k)
        eln_like_adverse = bool(is_mk or is_ck or has_minus5_or_del5q or has_minus7_or_del7q)

        def _prop(cond_fn):
            if total_meta_known == 0:
                return 0.0
            pos = sum(n_meta for _, n_meta, f in clone_info if n_meta and cond_fn(f))
            return pos / total_meta_known if total_meta_known else 0.0

        rows.append({
            'has_any_abnormality': int(any_abn),
            'n_events': int(n_events),
            'n_chromosomes_altered': int(n_chrs),
            'has_minus5_or_del5q': int(has_minus5_or_del5q),
            'has_minus7_or_del7q': int(has_minus7_or_del7q),
            'has_plus8': int(has_plus8),
            'has_t_8_21': int(has_t_8_21),
            'has_inv16_or_t_16_16': int(has_inv16_or_t_16_16),
            'has_t_15_17': int(has_t_15_17),
            'is_monosomal_karyotype': int(is_mk),
            'is_complex_karyotype': int(is_ck),
            'eln_like_flag_adverse_cyto': int(eln_like_adverse),
            'total_metaphases': int(total_meta_known),
            'prop_any_abnormal': float(_prop(lambda f: f['has_any_abn'])),
            'prop_adverse_5_7': float(_prop(lambda f: f['minus5_or_del5q'] or f['minus7_or_del7q'])),
            'prop_plus8': float(_prop(lambda f: f['plus8'])),
            'prop_favorable_core': float(_prop(lambda f: f['t_8_21'] or f['inv16_or_t_16_16'])),
        })
    features_df = pd.DataFrame(rows, index=df.index)
    return pd.concat([df.copy(), features_df], axis=1).drop(columns=[col])


In [25]:
X_enhanced = add_cytogenetics_features(X_w_mutation)
X_eval_enhanced = add_cytogenetics_features(X_eval_w_mutation)

# Identify nearly constant cytogenetics features (95% or more the same value)
cytogenetics_features = ['has_any_abnormality', 'n_events', 'n_chromosomes_altered',
                         'has_minus5_or_del5q', 'has_minus7_or_del7q', 'has_plus8',
                         'has_t_8_21', 'has_inv16_or_t_16_16', 'has_t_15_17',
                         'is_monosomal_karyotype', 'is_complex_karyotype',
                         'eln_like_flag_adverse_cyto', 'total_metaphases',
                         'prop_any_abnormal', 'prop_adverse_5_7', 'prop_plus8',
                         'prop_favorable_core']

nearly_constant_features = []
for col in cytogenetics_features:
    value_counts = X_enhanced[col].value_counts()
    if len(value_counts) > 0:
        max_proportion = value_counts.iloc[0] / len(X_enhanced)
        if max_proportion >= 0.95:
            nearly_constant_features.append(col)
            print(f"Removing {col}: {max_proportion:.2%} of values are {value_counts.index[0]}")

# Remove nearly constant features
X_enhanced = X_enhanced.drop(columns=nearly_constant_features)
X_eval_enhanced = X_eval_enhanced.drop(columns=nearly_constant_features)

# Keep only non-constant features for scaling
cytogenetics_features = [f for f in cytogenetics_features if f not in nearly_constant_features]

# Apply RobustScaler to the remaining cytogenetics features
cytogenetics_scaler = RobustScaler()
print(f"Fitting RobustScaler for cytogenetics features on training data, transforming training and evaluation data.")
X_enhanced[cytogenetics_features] = cytogenetics_scaler.fit_transform(X_enhanced[cytogenetics_features])
X_eval_enhanced[cytogenetics_features] = cytogenetics_scaler.transform(X_eval_enhanced[cytogenetics_features])


Removing has_t_8_21: 100.00% of values are 0
Removing has_inv16_or_t_16_16: 100.00% of values are 0
Removing has_t_15_17: 99.97% of values are 0
Removing prop_favorable_core: 100.00% of values are 0.0
Fitting RobustScaler for cytogenetics features on training data, transforming training and evaluation data.


In [26]:
df_enhanced = X_enhanced.merge(target_df, on='ID', how='left')
df_eval_enhanced = X_eval_enhanced


In [ ]:
import pandas as pd

def add_gene_features(df_clinical_enhanced, df_molecular, gene_list=None):
    """
    Ajoute des features one-hot pour les gènes :
    - Si gene_list est None : utilise tous les gènes présents dans df_molecular
    - Sinon : force l'usage de gene_list (pour que train et val aient les mêmes colonnes)
    """
    # 1) Définir la liste de gènes de référence
    if gene_list is None:
        gene_list = sorted(df_molecular['GENE'].unique().tolist())
        print(f"Nombre total de gènes utilisés : {len(gene_list)}")

    # 2) Ne garder que les lignes correspondant aux gènes de la liste
    df_filtered = df_molecular[df_molecular['GENE'].isin(gene_list)].copy()

    # 3) Table ID x GENE (one-hot)
    gene_pivot = pd.crosstab(df_filtered['ID'], df_filtered['GENE'])

    # 4) S'assurer que toutes les colonnes de gene_list existent (au cas où certaines
    #    n'apparaissent pas dans df_molecular pour ce split)
    for g in gene_list:
        if g not in gene_pivot.columns:
            gene_pivot[g] = 0

    # 5) Réordonner les colonnes exactement selon gene_list
    gene_pivot = gene_pivot[gene_list]

    # 6) Binariser et renommer les colonnes
    gene_pivot = (gene_pivot > 0).astype(int)
    gene_pivot.columns = [f'Gene_{col}' for col in gene_pivot.columns]

    # 7) Merge avec les features cliniques
    df_final = df_clinical_enhanced.merge(gene_pivot, on='ID', how='left')

    # 8) Remplir les NaN des colonnes de gènes avec 0
    new_cols = [c for c in df_final.columns if c.startswith('Gene_')]
    df_final[new_cols] = df_final[new_cols].fillna(0)

    return df_final, gene_list


# ======================================================================
# Lecture des données
# ======================================================================
mol_train_raw = pd.read_csv("../../data/molecular_train.csv")
mol_val_raw   = pd.read_csv("../../data/molecular_val.csv")

# df_enhanced et df_eval_enhanced sont tes jeux cliniques déjà préparés
# df_enhanced      : train clinique
# df_eval_enhanced : val clinique

# ======================================================================
# Construction des features gènes pour train et val
# ======================================================================
df_train_pivot, gene_list_ref = add_gene_features(
    df_clinical_enhanced=df_enhanced,
    df_molecular=mol_train_raw,
    gene_list=None  # on déduit tous les gènes du train
)

df_val_pivot, _ = add_gene_features(
    df_clinical_enhanced=df_eval_enhanced,
    df_molecular=mol_val_raw,
    gene_list=gene_list_ref  # on impose la même liste de gènes que le train
)

# ======================================================================
# Harmonisation des colonnes entre train et val
# ======================================================================

# Colonnes de gènes (identiques pour les deux par construction)
gene_cols = [c for c in df_train_pivot.columns if c.startswith('Gene_')]

# Colonnes cliniques pour les données d'entraînement (incluant OS_YEARS et OS_STATUS)
train_clinical_cols = [c for c in df_train_pivot.columns if not c.startswith('Gene_')]

# Colonnes cliniques pour les données de validation (excluant OS_YEARS et OS_STATUS)
val_clinical_cols = [c for c in df_val_pivot.columns if not c.startswith('Gene_')]

# Pour l'harmonisation, on exclut temporairement OS_YEARS et OS_STATUS
train_clinical_cols_for_common = [c for c in train_clinical_cols if c not in ['OS_YEARS', 'OS_STATUS']]

# On garde uniquement les colonnes cliniques communes (sans les targets)
common_clinical = sorted(set(train_clinical_cols_for_common).intersection(val_clinical_cols))

# Ordre final pour les données d'entraînement : colonnes cliniques communes + gènes + targets
final_features_train = common_clinical + gene_cols + ['OS_YEARS', 'OS_STATUS']

# Ordre final pour les données de validation : colonnes cliniques communes + gènes (pas de targets)
final_features_val = common_clinical + gene_cols

# Sélection des colonnes dans le bon ordre pour train et val
df_train_pivot = df_train_pivot[final_features_train]
df_val_pivot = df_val_pivot[final_features_val]

print(f"Nombre de colonnes finales train (avec targets) : {len(final_features_train)}")
print(f"Nombre de colonnes finales val (sans targets) : {len(final_features_val)}")
print(f"Shape train : {df_train_pivot.shape}")
print(f"Shape val   : {df_val_pivot.shape}")

# Vérification que OS_YEARS et OS_STATUS sont bien présents dans les données d'entraînement
print(f"OS_YEARS dans train : {'OS_YEARS' in df_train_pivot.columns}")
print(f"OS_STATUS dans train : {'OS_STATUS' in df_train_pivot.columns}")

# ======================================================================
# Sauvegarde
# ======================================================================
df_train_pivot.to_csv('../../data/train_pivot2.csv', index=False)
df_val_pivot.to_csv('../../data/eval_pivot2.csv',  index=False)

Nombre total de gènes utilisés : 124
Nombre de colonnes finales (features) : 183
Shape train : (3103, 183)
Shape val   : (1193, 183)
